## Deep Hedging of Long-Term Derivatives
This notebook presents an example of implementation of the deep hedging algorithm for the risk management of long-term contingent claims as presented in Carbonneau (2020).
- The deep hedging algorithm of Buehler et al. (2019) is applied to train neural networks to approximate optimal global hedging strategies.
- Results contained in this notebook replicate Table $5$ and $7$ of Carbonneau (2020) for the Black-Scholes dynamics.
    - Table 5: benchmarking of quadratic deep hedging (QDH) and semi-quadratic deep hedging (SQDH) with:
        - the underlying on a monthly and yearly basis;
        - two ATM yearly call and put options;
        - six yearly options (3 calls of moneynesses [1.0, 1.1, 1.2] and 3 puts of moneynesses [1.0, 0.9, 0.8]);
    - Table 7: computes the average equity exposure across the different hedging instruments;

- For a complete description of the algorithm, the reader is referred to section 3 of Carbonneau (2020).  

Important note: some parts of the code are inspired by the following implementation of the deep hedging algorithm: 
    - https://github.com/alexandrecarbonneau/Equal-risk-pricing-with-deep-hedging (from my paper Carbonneau and Godin (2020)).
    - https://nbviewer.jupyter.org/urls/people.math.ethz.ch/~jteichma/lecture_ml_web/lecture_3.ipynb 
    - https://github.com/mgroncki/DataScienceNotebooks/blob/master/DeepHedging/DeepHedging_Part1.ipynb 

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import datetime as dt
import numpy as np
np.seterr(divide = 'ignore') 
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from keras.layers.recurrent import LSTM
from keras.optimizers import Adam
from keras import backend as K
from scipy import stats
from scipy.stats import skew
from scipy.stats import norm

Using TensorFlow backend.


### 1) Deep hedging class
- This class implements the deep hedging algorithm of Buehler et al. (2019) with a long-short term memory (LSTM) as described in section 3.3 of Carbonneau (2020). 
- Can be applied for hedging with the underlying (monthly and yearly basis), with two options or with six options.

In [2]:
class DeepAgent(object):
    """
    Inputs:
    nbs_point_traj       : if [S_0,...,S_T], nbs_point_traj = T+1
    batch_size           : Size of the batch
    nbs_input            : Nbs of features of the neural networks
        - This is prior to adding V_{t} at each-timestep
    loss_type            : Loss function applied on the rewards {MSE, SMSE}
    nbs_assets           : Number of hedging instruments
    hidden_layers        : Number of LSTM cells
    nbs_units            : Fixed number of units per layer
    lr                   : Learning rate
    prepro_risky_assets  : preprocessing of risky asset prices {Log, Nothing}.
    hedging_instruments  : {"Stock", "ATM call and put", "Six options"}
    name                 : Model name to be saved
    freq_obs             : observation frequency of market {Monthly, Yearly}
    """
    def __init__(self, nbs_point_traj, batch_size, nbs_input, loss_type, nbs_assets, 
                 nbs_layers, nbs_units, lr, prepro_risky_assets, hedging_instruments, freq_obs, 
                 name='model'):
    
        tf.reset_default_graph()
        self.nbs_point_traj            = nbs_point_traj
        self.batch_size                = batch_size
        self.loss_type                 = loss_type
        self.nbs_assets                = nbs_assets
        self.nbs_layers                = nbs_layers
        self.nbs_units                 = nbs_units
        self.lr                        = lr
        self.hedging_instruments       = hedging_instruments
        self.freq_obs                  = freq_obs
            
        # 1) Placeholders - Stock price is normalized
        # self.input = [S_t, O_t,...,T-t]
        #   - S_t: price of the underlying (always included no mather what)
        #   - O_t: prices of options (not always included)
        self.input       = tf.placeholder(tf.float32, [nbs_point_traj, batch_size, nbs_input])  
        self.disc_tensor = tf.placeholder(tf.float32, [nbs_point_traj, batch_size, 1])   # To discount from 't' to 0.
        self.V_0         = tf.placeholder(tf.float32, [batch_size])                      # Initial portoflio value
        
        # Delta: number of shares held in each instrument
        self.strategy    = tf.zeros(shape = [nbs_point_traj-1, batch_size, nbs_assets], dtype=tf.float32)
    
        # 2) Unormalized risky asset prices
        if(self.hedging_instruments == "Stock"):
            self.unorm_risky_asset_price = self.inverse_processing(self.input[:,:,0], prepro_risky_assets) # all risky assets
            self.underlying_unorm_prices = self.unorm_risky_asset_price
        else:
            self.unorm_risky_asset_price = self.inverse_processing(self.input[:,:,:-1], prepro_risky_assets) 
            self.underlying_unorm_prices = self.unorm_risky_asset_price[:,:,0]    
        
        # 3) Unormalized hedging instrument prices
        # - self.unorm_hedging_inst_price_b: price process of the hedging instruments at the BEGINNING of each period 
        # - self.unorm_hedging_inst_price_e: price process of the hedging instruments at the END of each period
        if(self.nbs_assets ==1):
            if(self.hedging_instruments == "Stock"):
                self.unorm_hedging_inst_price_b = tf.expand_dims(self.underlying_unorm_prices[0:-1,:], axis=2)
                
                # For stock: end of price is the same as the beginning of the next period
                self.unorm_hedging_inst_price_e = tf.expand_dims(self.underlying_unorm_prices[1:,:], axis=2)

        elif(self.nbs_assets > 1):
            if(self.hedging_instruments == "ATM call and put" or self.hedging_instruments == "Six options"):
                self.unorm_hedging_inst_price_b = self.unorm_risky_asset_price[0:-1,:,1:]                    
                self.unorm_hedging_inst_price_e = self.payoff_liquid_inst_func()

        # 4) Discounted difference prices of the hedging instruments ONLY 
        unorm_discount_hedging_inst_price_b = self.unorm_hedging_inst_price_b*self.disc_tensor[0:-1,:,:]
        unorm_discount_hedging_inst_price_e = self.unorm_hedging_inst_price_e*self.disc_tensor[1:,:,:]
        inc_disc_ret                        = unorm_discount_hedging_inst_price_e - unorm_discount_hedging_inst_price_b
        
        # 5) Compute the hedging strategy for each time-step with the LSTM
        lstm = tf.contrib.cudnn_rnn.CudnnLSTM(num_layers = self.nbs_layers, num_units  = self.nbs_units)
            
        # 5.1) Output layer
        W_o = tf.get_variable(name='W_o', shape=[self.nbs_assets, self.nbs_units], initializer=tf.contrib.layers.xavier_initializer())
        b_o = tf.get_variable(name='b_o', shape=[self.nbs_assets], initializer=tf.constant_initializer(0.))

        # 5.2) Portfolio value - updated at each time-step
        V_t = self.V_0/self.V_0 # [batch_size]
            
        # 5.3) Compute hedging strategies for all time-steps
        for t in range(self.nbs_point_traj-1): 
            input_t = tf.expand_dims(tf.concat([self.input[t,:,:], tf.expand_dims(V_t, axis = 1)], axis=1), axis = 0)
            
            if(t==0):
                h1, final_state = lstm(input_t)
                    
                # With an LSTM, the output is of shape [1, BS, 1], no need to expand_dims self.strategy
                self.strategy   = tf.tensordot(h1, W_o, axes=[[2], [1]]) + b_o
                        
            else:
                h1, final_state = lstm(input_t, initial_state = final_state)
                output          = tf.tensordot(h1, W_o, axes=[[2], [1]]) + b_o

                # Store the resulting strategy in self.strategy
                self.strategy   = tf.concat([self.strategy, output], axis = 0)
     
            # Compute the value for the next time-period
            factor  = tf.div(self.disc_tensor[t,:,0],self.disc_tensor[t+1,:,0]) # equal exp(rh)   
            
            # Update hedging portfolio value
            V_t_pre = tf.multiply(self.V_0, V_t)
            V_t     = tf.divide(V_t_pre*factor + tf.reduce_sum(self.strategy[t,:,:]*(self.unorm_hedging_inst_price_e[t,:,:] - self.unorm_hedging_inst_price_b[t,:,:]*tf.expand_dims(factor, axis=1)), axis=1), self.V_0)
            
        # 6) Compute the payoff of the GMMB (i.e. lookback put option)
        # obs freq == yearly; payoff freq == yearly;
        if(self.freq_obs == 'Yearly'):
            self.payoff = tf.maximum(tf.reduce_max(self.underlying_unorm_prices[:-1,:],axis=0) - self.underlying_unorm_prices[-1,:],0)        
        
        # obs freq == monthly; payoff freq == yearly;
        elif(self.freq_obs == 'Monthly'):
            idx               = np.array(np.arange(0,int(10*12),12))  
            stock_yearly      = tf.gather(self.underlying_unorm_prices, idx)
            self.payoff       = tf.maximum(tf.reduce_max(stock_yearly,axis=0) - self.underlying_unorm_prices[-1,:],0)                    
        
        # 7) Compute hedging errors
        cumulative_factor  = tf.reciprocal(self.disc_tensor[-1,:,0])  # Project from 0 to T
        self.disc_gain     = tf.reduce_sum(inc_disc_ret*self.strategy, axis=[0,2])
        self.hedging_err   = self.payoff - cumulative_factor*(self.disc_gain + self.V_0)
        
        # 8) Compute the cost function (loss function) over the batch of hedging errors 
        if (self.loss_type == "MSE"):
            self.loss = tf.reduce_mean(tf.square(self.hedging_err))     
        elif (self.loss_type == "SMSE"):
            self.loss = tf.reduce_mean(tf.square(tf.nn.relu(self.hedging_err)))
        
        # 9) SGD step with the adam optimizer
        optimizer  = tf.train.AdamOptimizer(learning_rate = lr)  
        self.train = optimizer.minimize(self.loss)
        
        # 10) Save the model
        self.saver      = tf.train.Saver()
        self.model_name = name

    # Given a type of preprocessing for all risky assets, output the unormalized risky asset prices
    def inverse_processing(self, paths, prepro_risky_assets):
        if (prepro_risky_assets == "Log"):
            paths = tf.exp(paths)
        return paths
        
    def loss_out_optim(self, hedging_err, loss_type):
        if (loss_type == "MSE"):
            loss = np.mean(np.square(hedging_err)) 
        elif (loss_type == "SMSE"):
            loss = np.mean(np.square(np.where(hedging_err>0,hedging_err,0)))
        return loss

    # Compute payoff of options used as hedging instruments
    def payoff_liquid_inst_func(self):
        if(self.hedging_instruments == "ATM call and put"):
            # Two ATM call and put options at each time-step
            payoff_call = tf.expand_dims(tf.maximum(self.underlying_unorm_prices[1:,:] - self.underlying_unorm_prices[0:-1,:], 0), axis=2)
            payoff_put  = tf.expand_dims(tf.maximum(self.underlying_unorm_prices[0:-1,:] - self.underlying_unorm_prices[1:,:],0), axis=2)
            
            # Compilation
            payoff = tf.concat([payoff_call, payoff_put], axis=2) 
        
        elif(self.hedging_instruments == "Six options"):
            # 3 calls: [1.00, 1.1, 1.2]*S_t for the strike price at each time-step
            payoff_call_atm   = tf.expand_dims(tf.maximum(self.underlying_unorm_prices[1:,:] - self.underlying_unorm_prices[0:-1,:], 0), axis=2)
            payoff_call_OTM_1 = tf.expand_dims(tf.maximum(self.underlying_unorm_prices[1:,:] - 1.1*self.underlying_unorm_prices[0:-1,:], 0), axis=2)
            payoff_call_OTM_2 = tf.expand_dims(tf.maximum(self.underlying_unorm_prices[1:,:] - 1.2*self.underlying_unorm_prices[0:-1,:], 0), axis=2)
            
            # 3 puts : [1.00, 0.9, 0.8]*S_t for the strike price at each time-step
            payoff_put_atm   = tf.expand_dims(tf.maximum(self.underlying_unorm_prices[0:-1,:] - self.underlying_unorm_prices[1:,:],0), axis=2)
            payoff_put_OTM_1 = tf.expand_dims(tf.maximum(0.9*self.underlying_unorm_prices[0:-1,:] - self.underlying_unorm_prices[1:,:],0), axis=2)
            payoff_put_OTM_2 = tf.expand_dims(tf.maximum(0.8*self.underlying_unorm_prices[0:-1,:] - self.underlying_unorm_prices[1:,:],0), axis=2)
            
            # Compilation
            payoff = tf.concat([payoff_call_atm, payoff_call_OTM_1, payoff_call_OTM_2, payoff_put_atm, payoff_put_OTM_1, payoff_put_OTM_2], axis=2)            
        
        return(payoff)
    
    # ---------------------------------------------------------------------------------------# 
    # Function to call the deep hedging algorithm batch-wise
    # - Monitor performance on train and valid set on all epochs;
    # - Select parameter values which minimize loss on valid set;
    # Function adapted from https://github.com/mgroncki/DataScienceNotebooks/blob/master/DeepHedging/DeepHedging_Part1.ipynb 
    """
    Input:
     - paths_X     : tensor of features of dimension [n_timesteps+1, n_sims, nbs_features]
     - V_0         : vector of initial capital investment of dimension [N_batch]
         - could differ for each path, but for num. exp. always fixed
     - disc_batch  : tensor of discount factors of dimension [n_timesteps+1, N_batch, 1]
         - could fifer for each path, but for num. exp. always fixed
     - epochs      : total number of epochs to run
    """ 
    def train_deephedging(self, paths_train, V_0_batch, disc_batch, paths_valid, sess, epochs):
        
        sample_size_train = paths_train.shape[1]         
        sample_size_valid = paths_valid.shape[1]
        batch_size        = self.batch_size    
        idx_train         = np.arange(sample_size_train) 
        idx_valid         = np.arange(sample_size_valid)
        start             = dt.datetime.now()            
        self.loss_epochs  = 9999999*np.ones((epochs,2))  # store the loss at the end of each epoch for the train|valid 
        valid_loss_best   = 999999999                    # record best loss on valid set
        epoch             = 0 
        
        # 0) Loop while we haven't reached the max. epoch
        while (epoch < epochs):
            
            # Save the hedging errors for each batch      
            hedging_err_train = []
            hedging_err_valid = []
            np.random.shuffle(idx_train)  # randomize the dataset (useful if training set not already randomized)
            
            # 1) loop over sample size (train) to do one complete epoch
            for i in range(int(sample_size_train/batch_size)):
                
                # indexes of the paths for the batch
                indices = idx_train[i*batch_size : (i+1)*batch_size]

                # Training step
                _, hedging_err = sess.run([self.train, self.hedging_err], 
                                               {self.input        : paths_train[:,indices,:],
                                                self.V_0          : V_0_batch,
                                                self.disc_tensor  : disc_batch})
            
                # append hedging errors on train set for this batch
                hedging_err_train.append(hedging_err)
            
            # 2) Evaluate performance on the valid set - we don't train
            for i in range(int(sample_size_valid/batch_size)):
                indices_valid = idx_valid[i*batch_size : (i+1)*batch_size]                
                hedging_err_v = sess.run([self.hedging_err], 
                                            {self.input       : paths_valid[:,indices_valid,:],
                                            self.V_0          : V_0_batch,
                                            self.disc_tensor  : disc_batch})    
                    
                hedging_err_valid.append(hedging_err_v)
                        
            # 3) Store the loss on the train and valid sets after each epoch
            self.loss_epochs[epoch,0] = self.loss_out_optim(np.concatenate(hedging_err_train), self.loss_type)
            self.loss_epochs[epoch,1] = self.loss_out_optim(np.reshape(np.concatenate(hedging_err_valid, axis=1), sample_size_valid), 
                                                                self.loss_type)
                 
            # 4) Test if best epoch so far on valid set; if so, save model parameters.
            if(self.loss_epochs[epoch,1] < valid_loss_best):
                valid_loss_best = self.loss_epochs[epoch,1]
                self.saver.save(sess, r"/Users/alexa/Github files/Long-term hedging paper example/Models/%s/models.ckpt" % self.model_name)


            # 5) Print statistics during the optimization 
            if (epoch+1) % 1 == 0:
                print('Time elapsed:', dt.datetime.now()-start)
                print('Epoch %d, %s, Train: %.3f Valid: %.3f' % (epoch+1, self.loss_type, 
                                                            self.loss_epochs[epoch,0], self.loss_epochs[epoch,1]))   
            epoch+=1  # increment the epoch
                
        # 6) End of training
        print("---Finished training results---")
        print('Time elapsed:', dt.datetime.now()-start)    
  
        # 7) Return the loss per epoch on the train and valid set
        return self.loss_epochs
    
    # function to be called for training neural networks
    def training(self, paths_train, V_0_batch, disc_batch, paths_valid, sess, epochs, init=True):
        
        if init:
            # for the training part, you reset all of your parameters
            sess.run(tf.global_variables_initializer()) 
        loss_epoch = self.train_deephedging(paths_train, V_0_batch, disc_batch, paths_valid, sess, epochs)
        return loss_epoch
     
    # function to predict hedging strategies on test-set (no training)
    def predict(self, paths, V_0_batch, disc_batch, sess, loss_type):
        sample_size = paths.shape[1]
        batch_size=self.batch_size    
        idx = np.arange(sample_size)  
        start = dt.datetime.now()     
        
        # Save the hedging errors for each batch      
        hedging_err_pred = [] 
        strategy_pred = []
            
        for i in range(int(sample_size/batch_size)):
                
            # indexes of the paths of the batch
            indices               = idx[i*batch_size : (i+1)*batch_size]         
            
            # Compute hedging strategies for the batch  
            _, strategy = sess.run([self.hedging_err, self.strategy], 
                                                 {self.input        : paths[:,indices,:],
                                                  self.V_0          : V_0_batch,
                                                  self.disc_tensor  : disc_batch})  
            
            # append hedging positions for the batch
            strategy_pred.append(strategy)
            
        return np.concatenate(strategy_pred,axis=1)   
         
    def restore(self, sess, checkpoint):
        self.saver.restore(sess, checkpoint)

### 2) Construction of the train-valid-test sets for all cases (monthly|yearly underlying + two options + six options)
- A) Simulation of 500K paths under the Black-Scholes model with the parameters [sigma, mu] = [0.15, 0.10] on a yearly scale. 
- B) Compute option prices at each time-step for all paths
    - two options: ATM calls and puts;
    - six options: {S_t, 1.1S_t, 1.2S_t} for calls and {S_t, 0.9S_t, 0.8S_t} for puts
- C) Split into train-valid-test sets with 350K, 75K and 75K paths.
- D) Normalize with [log(S_n/S_0), log(Z_n/Z_0)] datasets

In [3]:
def BS_d1(S, dt, r, sigma, strike):
    return (np.log(S/strike) + (r+sigma**2/2)*dt) / (sigma*np.sqrt(dt))

# Style : +1 for call, -1 for put
def BlackScholes_price(S, T, r, sigma, strike, style, t=0):
    dt = T-t
    Phi = stats.norm(loc=0, scale=1).cdf
    d1 = BS_d1(S, dt, r, sigma, strike)
    d2 = d1 - sigma*np.sqrt(dt)
    return style*S*Phi(style*d1) - style*strike*np.exp(-r*dt)*Phi(style*d2)

# Style : +1 for call, -1 for put
def BS_delta(S, T, r, sigma, strike, style, t=0):
    dt = T-t
    d1 = BS_d1(S, dt, r, sigma, strike)
    Phi = stats.norm(loc=0, scale=1).cdf
    delta_call = Phi(d1)
    if(style == 1):
        result = delta_call
    else:
        result = delta_call - 1
    return result

In [4]:
# Parameters
[sigma, mu]        = [0.15, 0.10] # Yearly parameters
S_0                = 100     # Initial stock price
T                  = 10      # Time-to-maturity of the GMMB (lookback option)
n_sims             = 500000  # Total number of paths to simulate
n_timesteps_month  = 12*T    # Monthly time-steps
n_timesteps_year   = T       # Yearly time-steps
r                  = 0.03    # Annualized continuous risk-free rate

# A) Simulation of BSM dataset - monthly step-size
seed                 = 43
h                    = T / n_timesteps_month  # step-size
stdnorm_random_variates = np.random.randn(n_sims, n_timesteps_month)
Price_mat_month         = S_0 * np.cumprod(np.exp((mu-sigma**2/2)*h+sigma*np.sqrt(h)*stdnorm_random_variates), axis=1)
Price_mat_month         = np.reshape(np.transpose(np.c_[np.ones(n_sims)*S_0, Price_mat_month]), (n_timesteps_month+1, n_sims))

# B) Compute the Z_n process for monthly time-steps where Z_0 = S_0, Z_n = max(S_n, Z_n-1) for all time-steps 
Z_mat_month          = np.zeros((n_timesteps_month+1, n_sims))
Z_mat_month[0,:]     = S_0
for t in range(n_timesteps_month):
    # Need to update ONLY if we are at time t = {11,23,35,...,}
    if((t+1)%12==0):
        condition          = np.greater_equal(Price_mat_month[t+1,:], Z_mat_month[t,:])
        Z_mat_month[t+1,:] = np.where(condition, Price_mat_month[t+1,:], Z_mat_month[t,:])
    
    else:
        Z_mat_month[t+1,:] = Z_mat_month[t,:]
    
# C) Yearly step-size
idx            = np.arange(0,int(T*12) + 12,12)
Price_mat_year = np.zeros((n_timesteps_year+1,n_sims)) 
Price_mat_year = Price_mat_month[idx,:]

# D) Yearly Z_n process
Z_mat_year       = np.zeros((n_timesteps_year+1, n_sims))
Z_mat_year[0,:]  = S_0
for t in range(n_timesteps_year):
    condition          = np.greater_equal(Price_mat_year[t+1,:], Z_mat_year[t,:])
    Z_mat_year[t+1,:]  = np.where(condition, Price_mat_year[t+1,:], Z_mat_year[t,:])
    
# E) Price ATM yearly call and put options at each time-step for all paths
Opt_2_ATM_price          = np.zeros((n_timesteps_year+1, 500000,2))
Opt_2_ATM_price[:,:,0] = BlackScholes_price(Price_mat_year, 1, r, sigma, Price_mat_year, 1)  # Call option price
Opt_2_ATM_price[:,:,1] = BlackScholes_price(Price_mat_year, 1, r, sigma, Price_mat_year, -1) # Put option price

# F) Price six options 
# - Order: [C(K = S_t), C(K = 1.1S_t), C(K = 1.2S_t), P(K = S_t), P(K = 0.9S_t), P(K=0.8S_t)]
Opt_6_price          = np.zeros((n_timesteps_year+1, 500000,6))
Opt_6_price[:,:,0] = BlackScholes_price(Price_mat_year, 1, r, sigma, Price_mat_year, 1)      # ATM Call option price
Opt_6_price[:,:,1] = BlackScholes_price(Price_mat_year, 1, r, sigma, 1.1*Price_mat_year, 1)  # OTM Call option price
Opt_6_price[:,:,2] = BlackScholes_price(Price_mat_year, 1, r, sigma, 1.2*Price_mat_year, 1)  # DOTM Call option price
Opt_6_price[:,:,3] = BlackScholes_price(Price_mat_year, 1, r, sigma, Price_mat_year, -1)     # ATM Put option price
Opt_6_price[:,:,4] = BlackScholes_price(Price_mat_year, 1, r, sigma, 0.9*Price_mat_year, -1) # OTM Put option price
Opt_6_price[:,:,5] = BlackScholes_price(Price_mat_year, 1, r, sigma, 0.8*Price_mat_year, -1) # DOTM Put option price

# G) Split into train-valid-test sets
# G.1) Monthly underlying: [S_t, Z_t]
train_stock_month_input        = np.zeros((n_timesteps_month+1, 350000,2))
valid_stock_month_input        = np.zeros((n_timesteps_month+1, 75000,2))
test_stock_month_input         = np.zeros((n_timesteps_month+1, 75000,2))
train_stock_month_input[:,:,0] = Price_mat_month[:,0:350000]  
train_stock_month_input[:,:,1] = Z_mat_month[:,0:350000] 
valid_stock_month_input[:,:,0] = Price_mat_month[:,350000:425000]
valid_stock_month_input[:,:,1] = Z_mat_month[:,350000:425000] 
test_stock_month_input[:,:,0]  = Price_mat_month[:,425000:]
test_stock_month_input[:,:,1]  = Z_mat_month[:,425000:] 

# G.2) Yearly underlying: [S_t, Z_t]
train_stock_year_input        = np.zeros((n_timesteps_year+1, 350000,2))
valid_stock_year_input        = np.zeros((n_timesteps_year+1, 75000,2))
test_stock_year_input         = np.zeros((n_timesteps_year+1, 75000,2))
train_stock_year_input[:,:,0] = Price_mat_year[:,0:350000]  
train_stock_year_input[:,:,1] = Z_mat_year[:,0:350000] 
valid_stock_year_input[:,:,0] = Price_mat_year[:,350000:425000]
valid_stock_year_input[:,:,1] = Z_mat_year[:,350000:425000] 
test_stock_year_input[:,:,0]  = Price_mat_year[:,425000:]
test_stock_year_input[:,:,1]  = Z_mat_year[:,425000:]

# G.3) Two opts - [S_t, C_t, P_t, Z_t] where S_t = stock price, C_t = call price, P_t = put price
train_two_opts_input = np.zeros((n_timesteps_year+1, 350000,4))
valid_two_opts_input = np.zeros((n_timesteps_year+1, 75000,4))
test_two_opts_input  = np.zeros((n_timesteps_year+1, 75000,4))

train_two_opts_input[:,:,0]   = train_stock_year_input[:,:,0]
train_two_opts_input[:,:,1:3] = Opt_2_ATM_price[:,0:350000,:]
train_two_opts_input[:,:,-1]  = train_stock_year_input[:,:,-1]

valid_two_opts_input[:,:,0]   = valid_stock_year_input[:,:,0]
valid_two_opts_input[:,:,1:3] = Opt_2_ATM_price[:,350000:425000,:]
valid_two_opts_input[:,:,-1]  = valid_stock_year_input[:,:,-1]

test_two_opts_input[:,:,0]   = test_stock_year_input[:,:,0]
test_two_opts_input[:,:,1:3] = Opt_2_ATM_price[:,425000:,:]
test_two_opts_input[:,:,-1]  = test_stock_year_input[:,:,-1]

# G.4) Six opts - [S_t, C_t, 1.1C_t, 1.2C_t, P_t, 0.9P_t, 0.8P_t, Z_t]
train_six_opts_input = np.zeros((n_timesteps_year+1, 350000,8))
valid_six_opts_input = np.zeros((n_timesteps_year+1, 75000,8))
test_six_opts_input  = np.zeros((n_timesteps_year+1, 75000,8))

train_six_opts_input[:,:,0]   = train_stock_year_input[:,:,0]
train_six_opts_input[:,:,1:7] = Opt_6_price[:,0:350000,:]
train_six_opts_input[:,:,-1]  = train_stock_year_input[:,:,-1]

valid_six_opts_input[:,:,0]   = valid_stock_year_input[:,:,0]
valid_six_opts_input[:,:,1:7] = Opt_6_price[:,350000:425000,:]
valid_six_opts_input[:,:,-1]  = valid_stock_year_input[:,:,-1]

test_six_opts_input[:,:,0]   = test_stock_year_input[:,:,0]
test_six_opts_input[:,:,1:7] = Opt_6_price[:,425000:,:]
test_six_opts_input[:,:,-1]  = test_stock_year_input[:,:,-1]

# Preprocessing of all risky aset prices as {log(S_n/S_0), log(Z_n/S_0)}
# - Also store initial risky asset prices (i.e. at time-0) for all cases
prepro_risky_assets = "Log"
if (prepro_risky_assets == "Log"):
    train_stock_month_input = np.log(train_stock_month_input)
    valid_stock_month_input = np.log(valid_stock_month_input)
    test_stock_month_input  = np.log(test_stock_month_input)
    
    train_stock_year_input = np.log(train_stock_year_input)
    valid_stock_year_input = np.log(valid_stock_year_input)
    test_stock_year_input  = np.log(test_stock_year_input)
    
    train_two_opts_input = np.log(train_two_opts_input)
    valid_two_opts_input = np.log(valid_two_opts_input)
    test_two_opts_input  = np.log(test_two_opts_input)
    
    train_six_opts_input = np.log(train_six_opts_input)
    valid_six_opts_input = np.log(valid_six_opts_input)
    test_six_opts_input  = np.log(test_six_opts_input)

### 3) Hyperparameters of neural networks as in Section 4.1.2 of the paper
- Initial portfolio value under the BSM model: 17.68894582328168

In [5]:
batch_size    = 1000   # batch size
epochs        = 150    # number of epochs
hidden_layers = 2      # number of hidden layers (output layer not included, for a total of three layers)
nbs_units     = 24     # neurons per layer  
price         = 17.68894582328168  # Initial portfolio value
V_0_train     = np.ones(train_stock_year_input.shape[1])*price  # initial portfolio value
V_0_valid     = np.ones(valid_stock_year_input.shape[1])*price
V_0_test      = np.ones(test_stock_year_input.shape[1])*price
V_0_batch     = np.ones(batch_size)*price

### 4) Tensor of discount factors
- disc_mat: (n_timesteps+1 x n_sims x 1) tensor of cumulative discount rates (discount from t to 0), i.e. $\exp(-rh*n)$ 
    - Will be split into disc_mat_train and disc_mat_test for convenience

In [6]:
# Monthly tensors
disc_mat_month              = np.zeros((n_timesteps_month+1, 500000, 1))
disc_train_month            = np.zeros((n_timesteps_month+1, 350000, 1))
disc_valid_month            = np.zeros((n_timesteps_month+1, 75000, 1))
disc_test_month             = np.zeros((n_timesteps_month+1, 75000, 1))
discount_vect_month         = np.ones(disc_mat_month.shape[0])   
h                           = T / n_timesteps_month
discount_vect_month[1:]     = np.exp(-r*h)              # [1,exp(-rh), exp(-rh),....,exp(-rh)]
discount_vect_month         = np.cumprod(discount_vect_month) # [1,exp(-rh), exp(-r2h),....,exp(-rNh)]
disc_mat_month              = np.reshape(np.repeat(discount_vect_month, 500000), (n_timesteps_month+1, 500000,1))
disc_train_month            = disc_mat_month[:,0:350000,:]
disc_valid_month            = disc_mat_month[:,350000:425000,:]
disc_test_month             = disc_mat_month[:,425000::,:]
disc_batch_month            = disc_mat_month[:,0:batch_size,:]  # for convenience when only batch-size is needed
disc_mat_month              = []

# yearly tensors
disc_mat_year              = np.zeros((n_timesteps_year+1, 500000, 1))
disc_train_year            = np.zeros((n_timesteps_year+1, 350000, 1))
disc_valid_year            = np.zeros((n_timesteps_year+1, 75000, 1))
disc_test_year             = np.zeros((n_timesteps_year+1, 75000, 1))
discount_vect_year         = np.ones(disc_mat_year.shape[0])   
h                           = T / n_timesteps_year
discount_vect_year[1:]     = np.exp(-r*h)              # [1,exp(-rh), exp(-rh),....,exp(-rh)]
discount_vect_year         = np.cumprod(discount_vect_year) # [1,exp(-rh), exp(-r2h),....,exp(-rNh)]
disc_mat_year              = np.reshape(np.repeat(discount_vect_year, 500000), (n_timesteps_year+1, 500000,1))
disc_train_year            = disc_mat_year[:,0:350000,:]
disc_valid_year            = disc_mat_year[:,350000:425000,:]
disc_test_year             = disc_mat_year[:,425000::,:]
disc_batch_year            = disc_mat_year[:,0:batch_size,:]  # for convenience when only batch-size is needed
disc_mat_year              = []

### 5) Function to compute hedging statistics as in Table 3 of the paper

In [7]:
#-------------------------------------------------------------------------------------------------#
#---------------         Functions to compute the hedging statistics         ---------------------#
#-------------------------------------------------------------------------------------------------#
"""
deltas              : delta strategy path-wise
paths               : risky asset prices path-wise 
disc_paths          : discount factors path-wise
hedging_instruments : {"Stock", "ATM call and put", "Six options"}
prepro_risky_assets : which pre-processing for risky assets {"Log", "Nothing"}
V_0                 : initial price investment, can be a vector of the dimension for each paths
freq_obs            : {'Monthly', 'Yearly'} 
"""
def hedging_stats(deltas, paths, disc_paths, V_0, loss_type, model_name, prepro_risky_assets, hedging_instruments, 
                  freq_obs):
    # 1) Number of hedging instruments
    nbs_assets = deltas.shape[2]

    # 2) Store prices of the underlying unormalized
    if(hedging_instruments == "Stock"):
        unorm_risky_asset_price = inverse_processing(paths[:,:,0], prepro_risky_assets) # all risky assets
        underlying_unorm_prices = unorm_risky_asset_price
    else:
        # all risky assets except Z_{n} process
        unorm_risky_asset_price = inverse_processing(paths[:,:,0:-1], prepro_risky_assets) 
        underlying_unorm_prices = unorm_risky_asset_price[:,:,0]   

    # 3) Store the unormalized hedging instrument price beginning and ending values (see paper section 2)
    if(hedging_instruments == "Stock"):
        unorm_hedging_inst_price_b = np.expand_dims(underlying_unorm_prices[0:-1,:], axis=2)
        unorm_hedging_inst_price_e = np.expand_dims(underlying_unorm_prices[1:,:], axis=2)

    elif(hedging_instruments == "ATM call and put" or hedging_instruments == "Six options"):
        unorm_hedging_inst_price_b = unorm_risky_asset_price[0:-1,:,1:]
        unorm_hedging_inst_price_e = payoff_liquid_inst_func(hedging_instruments, underlying_unorm_prices, nbs_assets)

    # 4) Store the unormalized hedging instrument prices discounted at time "0"    
    unorm_discount_hedging_inst_price_b = unorm_hedging_inst_price_b*disc_paths[0:-1,:,:]
    unorm_discount_hedging_inst_price_e = unorm_hedging_inst_price_e*disc_paths[1:,:,:]
    inc_disc_ret                        = unorm_discount_hedging_inst_price_e - unorm_discount_hedging_inst_price_b
        
    # 5) Compute the payoff of the lookback option to hedge
    if(freq_obs == "Yearly"):
        liability_payoff = np.maximum(np.amax(underlying_unorm_prices[:-1,:],axis=0) - underlying_unorm_prices[-1,:],0)
    elif(freq_obs == "Monthly"):
        idx              = np.array(np.arange(0,int(10*12),12))
        liability_payoff = np.maximum(np.amax(underlying_unorm_prices[idx,:],axis=0) - underlying_unorm_prices[-1,:],0)
    else:
        print("There's an error somehwere!")
    
    # 6) Compute the hedging error
    # There's an ordering in the double np.sum:
    # - np.sum(deltas*inc_disc_ret, axis = 0).shape = [nbs of paths x nbs_assets], you sum over time-steps
    # - np.sum(np.sum(deltas*inc_disc_ret, axis = 0), axis = 1), you sum afterwards over the assets
    disc_hedging_gain = np.sum(np.sum(deltas*inc_disc_ret, axis = 0), axis = 1) # as of T
    cumulative_factor = np.reciprocal(disc_paths[-1,:,0]) 
    hedging_err       = liability_payoff - cumulative_factor*(disc_hedging_gain + V_0)
    
    # 7) Compute hedging statistics as presented in Table 3 of paper
    mean_hedging_err = np.mean(hedging_err)
    CVaR_95          = np.mean(np.sort(hedging_err)[int(0.95*hedging_err.shape[0]):])
    CVaR_99          = np.mean(np.sort(hedging_err)[int(0.99*hedging_err.shape[0]):])
    VaR_95           = np.sort(hedging_err)[int(0.95*hedging_err.shape[0])]
    VaR_99           = np.sort(hedging_err)[int(0.99*hedging_err.shape[0])]
    MSE              = np.mean(np.square(hedging_err))
    semi_MSE         = np.mean(np.square(np.where(hedging_err>0,hedging_err,0)))
    skew_            = skew(hedging_err)
    
    # 8) Present hedging statistics
    if (loss_type == "MSE"):
        loss = MSE
    elif (loss_type == "SMSE"):
        loss = semi_MSE
    print('Model was trained with the loss function: %s' %(loss_type))
    print('Initial investment', V_0[0])
    print('Mean Hedging error:', mean_hedging_err)
    print('CVaR_95: %.4f, CVaR_99: %.4f' % (CVaR_95, CVaR_99))
    print('VaR_95: %.4f, VaR_99: %.4f' % (VaR_95, VaR_99))
    print('MSE: %.4f, RMSE: %.4f' % (MSE, np.sqrt(MSE)))
    print('Semi-MSE: %.4f, Semi-RMSE: %.4f' %(semi_MSE, np.sqrt(semi_MSE)))
    print('Skew: %.4f' %(skew_))
    
def payoff_liquid_inst_func(hedging_instruments, underlying_unorm_prices, nbs_assets): 
    if (hedging_instruments == "ATM call and put"):
        payoff_call = np.expand_dims(np.maximum(underlying_unorm_prices[1:,:] - underlying_unorm_prices[0:-1,:], 0), axis=2)
        payoff_put = np.expand_dims(np.maximum(underlying_unorm_prices[0:-1,:] - underlying_unorm_prices[1:,:], 0), axis=2)

        # Compilation
        payoff = np.concatenate([payoff_call, payoff_put], axis=2)
        
    elif(hedging_instruments == "Six options"):
        # 3 calls: [1.00, 1.1, 1.2]*S_t for the strike price at each time-step
        payoff_call_atm   = np.expand_dims(np.maximum(underlying_unorm_prices[1:,:] - underlying_unorm_prices[0:-1,:], 0), axis=2)
        payoff_call_OTM_1 = np.expand_dims(np.maximum(underlying_unorm_prices[1:,:] - 1.1*underlying_unorm_prices[0:-1,:], 0), axis=2)
        payoff_call_OTM_2 = np.expand_dims(np.maximum(underlying_unorm_prices[1:,:] - 1.2*underlying_unorm_prices[0:-1,:], 0), axis=2)

        # 3 puts : [1.00, 0.9, 0.8]*S_t for the strike price at each time-step
        payoff_put_atm   = np.expand_dims(np.maximum(underlying_unorm_prices[0:-1,:] - underlying_unorm_prices[1:,:],0), axis=2)
        payoff_put_OTM_1 = np.expand_dims(np.maximum(0.9*underlying_unorm_prices[0:-1,:] - underlying_unorm_prices[1:,:],0), axis=2)
        payoff_put_OTM_2 = np.expand_dims(np.maximum(0.8*underlying_unorm_prices[0:-1,:] - underlying_unorm_prices[1:,:],0), axis=2)

        # Compilation
        payoff = np.concatenate([payoff_call_atm, payoff_call_OTM_1, payoff_call_OTM_2, payoff_put_atm, payoff_put_OTM_1, payoff_put_OTM_2], axis=2)

    return(payoff)

def inverse_processing(paths, prepro_risky_assets):
    if (prepro_risky_assets == "Log"):
        paths = np.exp(paths)
    return paths

### 6) Training of LSTM - Monthly underlying

In [8]:
# Normalization 
nbs_assets                = 1
hedging_instruments       = "Stock"
freq_obs                  = "Monthly"

# A) Part 1: MSE
loss_type  = 'MSE'
model_name = 'Monthly_underlying_MSE'
lr         = 0.01   # learning rate of the Adam optimizer
print("Hyperparameters: learning rate=%.4f, LSTM cells=%d, neurons=%d" %(lr, hidden_layers, nbs_units))
LSTM_underlying = DeepAgent(train_stock_month_input.shape[0], batch_size, train_stock_month_input.shape[2], 
        loss_type, nbs_assets, hidden_layers, nbs_units, lr, prepro_risky_assets, hedging_instruments,
        freq_obs, name = model_name)
    
# Start training 
start = dt.datetime.now()
print('---Training for quadratic deep hedging start---')
with tf.Session() as sess:           
    loss_epoch = LSTM_underlying.training(train_stock_month_input, V_0_batch, disc_batch_month, valid_stock_month_input, 
                                          sess, epochs)
    print('---Training for quadratic deep hedging end---')
    
# Compute measured risk exposure on train, valid and test set with the optimized LSTM
model_predict = DeepAgent(test_stock_month_input.shape[0], batch_size, test_stock_month_input.shape[2], loss_type, 
        nbs_assets, hidden_layers, nbs_units, lr, prepro_risky_assets, hedging_instruments, freq_obs, 
        name = model_name)

with tf.Session() as sess:
    
    # Load the saved model
    model_predict.restore(sess, r"/Users/alexa/Github files/Long-term hedging paper example/Models/%s/models.ckpt" % model_name)

    print("-------------------------------")
    print("Results on the TRAIN SET:")
    # Compute hedging strategies on the training set
    deltas = model_predict.predict(train_stock_month_input, V_0_batch, disc_batch_month, sess, loss_type)    
    hedging_stats(deltas, train_stock_month_input, disc_train_month, V_0_train, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
    print("-------------------------------")
    print("Results on the VALID SET:")
    # Compute hedging strategies on the valid set
    deltas = model_predict.predict(valid_stock_month_input, V_0_batch, disc_batch_month, sess, loss_type)
    hedging_stats(deltas, valid_stock_month_input, disc_valid_month, V_0_valid, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
    print("-------------------------------")
    print("Results on the TEST SET:")
    # Compute hedging strategies on the test set
    deltas_MSE = model_predict.predict(test_stock_month_input, V_0_batch, disc_batch_month, sess, loss_type)
    hedging_stats(deltas_MSE, test_stock_month_input, disc_test_month, V_0_test, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)

print("-------------------------------")
print("-------------------------------")
print("END OF QUADRATIC DEEP HEDGING")
print("-------------------------------")
print("-------------------------------")

# --------------------------------------------------- #
# B) Part 2: SMSE
loss_type  = 'SMSE'
model_name = 'Monthly_underlying_SMSE'
lr         = 0.01/6   # learning rate of the Adam optimizer
print("Hyperparameters: learning rate=%.4f, LSTM cells=%d, neurons=%d" %(lr, hidden_layers, nbs_units))
LSTM_underlying = DeepAgent(train_stock_month_input.shape[0], batch_size, train_stock_month_input.shape[2], 
        loss_type, nbs_assets, hidden_layers, nbs_units, lr, prepro_risky_assets, hedging_instruments, 
        freq_obs, name = model_name)
    
# Start training 
start = dt.datetime.now()
print('---Training for semi-quadratic deep hedging start---')
with tf.Session() as sess:           
    loss_epoch = LSTM_underlying.training(train_stock_month_input, V_0_batch, disc_batch_month, valid_stock_month_input, 
                                          sess, epochs)
    print('---Training for semi-quadratic deep hedging end---')
    
# Compute measured risk exposure on train, valid and test set with the optimized LSTM
model_predict = DeepAgent(test_stock_month_input.shape[0], batch_size, test_stock_month_input.shape[2], loss_type, 
        nbs_assets, hidden_layers, nbs_units, lr, prepro_risky_assets, hedging_instruments, freq_obs, 
        name = model_name)

with tf.Session() as sess:
    
    # Load the saved model
    model_predict.restore(sess, r"/Users/alexa/Github files/Long-term hedging paper example/Models/%s/models.ckpt" % model_name)

    print("-------------------------------")
    print("Results on the TRAIN SET:")
    # Compute hedging strategies on the training set
    deltas = model_predict.predict(train_stock_month_input, V_0_batch, disc_batch_month, sess, loss_type)    
    hedging_stats(deltas, train_stock_month_input, disc_train_month, V_0_train, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
    print("-------------------------------")
    print("Results on the VALID SET:")
    # Compute hedging strategies on the valid set
    deltas = model_predict.predict(valid_stock_month_input, V_0_batch, disc_batch_month, sess, loss_type)
    hedging_stats(deltas, valid_stock_month_input, disc_valid_month, V_0_valid, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
    print("-------------------------------")
    print("Results on the TEST SET:")
    # Compute hedging strategies on the test set
    deltas_SMSE = model_predict.predict(test_stock_month_input, V_0_batch, disc_batch_month, sess, loss_type)
    hedging_stats(deltas_SMSE, test_stock_month_input, disc_test_month, V_0_test, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
    
# Compute average equity risk exposure
stock_price_unorm = np.exp(test_stock_month_input[:,:,0])
portfolio_exposure_QDH  = deltas_MSE
portfolio_exposure_SQDH = deltas_SMSE
print(" ----------------------- ")
print("Average exposure QDH: %.4f" %(np.mean(portfolio_exposure_QDH)))
print(" ----------------------- ")
print("Average exposure SQDH: %.4f" %(np.mean(portfolio_exposure_SQDH)))

Hyperparameters: learning rate=0.0100, LSTM cells=2, neurons=24
---Training for quadratic deep hedging start---
Time elapsed: 0:00:56.231877
Epoch 1, MSE, Train: 369.616 Valid: 223.829
Time elapsed: 0:01:42.018264
Epoch 2, MSE, Train: 109.583 Valid: 66.962
Time elapsed: 0:02:27.805683
Epoch 3, MSE, Train: 59.468 Valid: 58.846
Time elapsed: 0:03:13.751107
Epoch 4, MSE, Train: 51.970 Valid: 44.899
Time elapsed: 0:03:57.979132
Epoch 5, MSE, Train: 63.148 Valid: 46.237
Time elapsed: 0:04:43.510471
Epoch 6, MSE, Train: 47.304 Valid: 40.548
Time elapsed: 0:05:28.275626
Epoch 7, MSE, Train: 44.994 Valid: 63.252
Time elapsed: 0:06:14.155027
Epoch 8, MSE, Train: 44.892 Valid: 39.960
Time elapsed: 0:06:59.558368
Epoch 9, MSE, Train: 44.228 Valid: 38.339
Time elapsed: 0:07:45.200723
Epoch 10, MSE, Train: 41.342 Valid: 37.697
Time elapsed: 0:08:29.202706
Epoch 11, MSE, Train: 39.515 Valid: 47.230
Time elapsed: 0:09:14.366952
Epoch 12, MSE, Train: 38.321 Valid: 35.272
Time elapsed: 0:10:00.211353
E

Time elapsed: 1:24:52.288209
Epoch 114, MSE, Train: 29.922 Valid: 29.260
Time elapsed: 1:25:38.468253
Epoch 115, MSE, Train: 29.934 Valid: 27.865
Time elapsed: 1:26:24.605389
Epoch 116, MSE, Train: 28.977 Valid: 28.217
Time elapsed: 1:27:11.589194
Epoch 117, MSE, Train: 29.492 Valid: 30.378
Time elapsed: 1:27:57.160691
Epoch 118, MSE, Train: 29.449 Valid: 31.142
Time elapsed: 1:28:42.656303
Epoch 119, MSE, Train: 29.211 Valid: 30.368
Time elapsed: 1:29:28.732899
Epoch 120, MSE, Train: 28.836 Valid: 29.580
Time elapsed: 1:30:14.926094
Epoch 121, MSE, Train: 30.676 Valid: 28.069
Time elapsed: 1:30:59.807294
Epoch 122, MSE, Train: 29.578 Valid: 31.631
Time elapsed: 1:31:44.935813
Epoch 123, MSE, Train: 28.611 Valid: 32.399
Time elapsed: 1:32:30.337113
Epoch 124, MSE, Train: 29.101 Valid: 27.269
Time elapsed: 1:33:16.008481
Epoch 125, MSE, Train: 28.597 Valid: 31.461
Time elapsed: 1:34:01.780873
Epoch 126, MSE, Train: 28.740 Valid: 27.789
Time elapsed: 1:34:47.016136
Epoch 127, MSE, Train:

Time elapsed: 0:44:02.075725
Epoch 58, SMSE, Train: 2.442 Valid: 2.485
Time elapsed: 0:44:47.531047
Epoch 59, SMSE, Train: 2.898 Valid: 2.475
Time elapsed: 0:45:32.489531
Epoch 60, SMSE, Train: 2.304 Valid: 2.588
Time elapsed: 0:46:16.373479
Epoch 61, SMSE, Train: 2.257 Valid: 2.535
Time elapsed: 0:47:01.739780
Epoch 62, SMSE, Train: 2.424 Valid: 2.289
Time elapsed: 0:47:45.878794
Epoch 63, SMSE, Train: 2.368 Valid: 2.382
Time elapsed: 0:48:30.364887
Epoch 64, SMSE, Train: 2.214 Valid: 2.568
Time elapsed: 0:49:15.646152
Epoch 65, SMSE, Train: 2.343 Valid: 2.207
Time elapsed: 0:49:59.764170
Epoch 66, SMSE, Train: 12.958 Valid: 4.284
Time elapsed: 0:50:44.044216
Epoch 67, SMSE, Train: 3.282 Valid: 2.863
Time elapsed: 0:51:28.079205
Epoch 68, SMSE, Train: 2.813 Valid: 3.176
Time elapsed: 0:52:12.054182
Epoch 69, SMSE, Train: 2.711 Valid: 2.551
Time elapsed: 0:52:56.224196
Epoch 70, SMSE, Train: 2.507 Valid: 2.669
Time elapsed: 0:53:40.700294
Epoch 71, SMSE, Train: 2.504 Valid: 2.523
Time 

### 7) Training of LSTM - Yearly underlying
This computes the hedging statistics as presented in Table 3 of the paper for under the penalties MSE and SMSE with the underlying on a yearly basis

In [9]:
nbs_assets          = 1
hedging_instruments = "Stock"
freq_obs            = "Yearly"

# A) Part 1: MSE
loss_type  = 'MSE'
model_name = 'Yearly_underlying_MSE'
lr         = 0.01   # learning rate of the Adam optimizer
print("Hyperparameters: learning rate=%.4f, LSTM cells=%d, neurons=%d" %(lr, hidden_layers, nbs_units))
LSTM_underlying = DeepAgent(train_stock_year_input.shape[0], batch_size, train_stock_year_input.shape[2], 
        loss_type, nbs_assets, hidden_layers, nbs_units, lr, prepro_risky_assets, hedging_instruments, 
        freq_obs, name = model_name)
    
# Start training 
start = dt.datetime.now()
print('---Training for quadratic deep hedging start---')
with tf.Session() as sess:           
    loss_epoch = LSTM_underlying.training(train_stock_year_input, V_0_batch, disc_batch_year, valid_stock_year_input, 
                                          sess, epochs)
    print('---Training for quadratic deep hedging end---')
    
# Compute measured risk exposure on train, valid and test set with the optimized LSTM
model_predict = DeepAgent(test_stock_year_input.shape[0], batch_size, test_stock_year_input.shape[2], loss_type, 
        nbs_assets, hidden_layers, nbs_units, lr, prepro_risky_assets, hedging_instruments, 
        freq_obs, name = model_name)

with tf.Session() as sess:
    
    # Load the saved model
    model_predict.restore(sess, r"/Users/alexa/Github files/Long-term hedging paper example/Models/%s/models.ckpt" % model_name)

    print("-------------------------------")
    print("Results on the TRAIN SET:")
    # Compute hedging strategies on the training set
    deltas = model_predict.predict(train_stock_year_input, V_0_batch, disc_batch_year, sess, loss_type)    
    hedging_stats(deltas, train_stock_year_input, disc_train_year, V_0_train, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
    print("-------------------------------")
    print("Results on the VALID SET:")
    # Compute hedging strategies on the valid set
    deltas = model_predict.predict(valid_stock_year_input, V_0_batch, disc_batch_year, sess, loss_type)
    hedging_stats(deltas, valid_stock_year_input, disc_valid_year, V_0_valid, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
    print("-------------------------------")
    print("Results on the TEST SET:")
    # Compute hedging strategies on the test set
    deltas_MSE = model_predict.predict(test_stock_year_input, V_0_batch, disc_batch_year, sess, loss_type)
    hedging_stats(deltas_MSE, test_stock_year_input, disc_test_year, V_0_test, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)

print("-------------------------------")
print("-------------------------------")
print("END OF QUADRATIC DEEP HEDGING")
print("-------------------------------")
print("-------------------------------")

# --------------------------------------------------- #
# B) Part 2: SMSE
loss_type  = 'SMSE'
model_name = 'Yearly_underlying_SMSE'
lr         = 0.01/6   # learning rate of the Adam optimizer
print("Hyperparameters: learning rate=%.4f, LSTM cells=%d, neurons=%d" %(lr, hidden_layers, nbs_units))
LSTM_underlying = DeepAgent(train_stock_year_input.shape[0], batch_size, train_stock_year_input.shape[2], 
        loss_type, nbs_assets, hidden_layers, nbs_units, lr, prepro_risky_assets, hedging_instruments, 
        freq_obs, name = model_name)
    
# Start training 
start = dt.datetime.now()
print('---Training for semi-quadratic deep hedging start---')
with tf.Session() as sess:           
    loss_epoch = LSTM_underlying.training(train_stock_year_input, V_0_batch, disc_batch_year, valid_stock_year_input, 
                                          sess, epochs)
    print('---Training for semi-quadratic deep hedging end---')
    
# Compute measured risk exposure on train, valid and test set with the optimized LSTM
model_predict = DeepAgent(test_stock_year_input.shape[0], batch_size, test_stock_year_input.shape[2], loss_type, 
        nbs_assets, hidden_layers, nbs_units, lr, prepro_risky_assets, hedging_instruments, 
        freq_obs, name = model_name)

with tf.Session() as sess:
    
    # Load the saved model
    model_predict.restore(sess, r"/Users/alexa/Github files/Long-term hedging paper example/Models/%s/models.ckpt" % model_name)

    print("-------------------------------")
    print("Results on the TRAIN SET:")
    # Compute hedging strategies on the training set
    deltas = model_predict.predict(train_stock_year_input, V_0_batch, disc_batch_year, sess, loss_type)    
    hedging_stats(deltas, train_stock_year_input, disc_train_year, V_0_train, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
    print("-------------------------------")
    print("Results on the VALID SET:")
    # Compute hedging strategies on the valid set
    deltas = model_predict.predict(valid_stock_year_input, V_0_batch, disc_batch_year, sess, loss_type)
    hedging_stats(deltas, valid_stock_year_input, disc_valid_year, V_0_valid, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
    print("-------------------------------")
    print("Results on the TEST SET:")
    # Compute hedging strategies on the test set
    deltas_SMSE = model_predict.predict(test_stock_year_input, V_0_batch, disc_batch_year, sess, loss_type)
    hedging_stats(deltas_SMSE, test_stock_year_input, disc_test_year, V_0_test, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
# Compute average equity risk exposure
stock_price_unorm = np.exp(test_stock_year_input[:,:,0])
portfolio_exposure_QDH  = deltas_MSE
portfolio_exposure_SQDH = deltas_SMSE
print(" ----------------------- ")
print("Average exposure QDH: %.4f" %(np.mean(portfolio_exposure_QDH)))
print(" ----------------------- ")
print("Average exposure SQDH: %.4f" %(np.mean(portfolio_exposure_SQDH)))

Hyperparameters: learning rate=0.0100, LSTM cells=2, neurons=24
---Training for quadratic deep hedging start---
Time elapsed: 0:00:05.032142
Epoch 1, MSE, Train: 319.718 Valid: 250.473
Time elapsed: 0:00:09.282114
Epoch 2, MSE, Train: 246.148 Valid: 248.483
Time elapsed: 0:00:13.505073
Epoch 3, MSE, Train: 235.294 Valid: 233.642
Time elapsed: 0:00:17.746035
Epoch 4, MSE, Train: 229.767 Valid: 227.711
Time elapsed: 0:00:21.826962
Epoch 5, MSE, Train: 226.734 Valid: 231.273
Time elapsed: 0:00:26.071947
Epoch 6, MSE, Train: 229.078 Valid: 227.491
Time elapsed: 0:00:30.319911
Epoch 7, MSE, Train: 226.813 Valid: 226.563
Time elapsed: 0:00:34.568875
Epoch 8, MSE, Train: 226.701 Valid: 226.118
Time elapsed: 0:00:38.805836
Epoch 9, MSE, Train: 224.563 Valid: 224.736
Time elapsed: 0:00:42.895773
Epoch 10, MSE, Train: 223.871 Valid: 225.845
Time elapsed: 0:00:46.997704
Epoch 11, MSE, Train: 223.308 Valid: 228.116
Time elapsed: 0:00:51.122640
Epoch 12, MSE, Train: 223.038 Valid: 229.258
Time elap

Time elapsed: 0:07:36.686766
Epoch 111, MSE, Train: 216.540 Valid: 220.467
Time elapsed: 0:07:40.770695
Epoch 112, MSE, Train: 216.460 Valid: 219.617
Time elapsed: 0:07:44.879625
Epoch 113, MSE, Train: 216.194 Valid: 221.313
Time elapsed: 0:07:48.959567
Epoch 114, MSE, Train: 216.097 Valid: 220.103
Time elapsed: 0:07:53.026473
Epoch 115, MSE, Train: 216.665 Valid: 220.854
Time elapsed: 0:07:57.124412
Epoch 116, MSE, Train: 216.468 Valid: 223.196
Time elapsed: 0:08:01.212339
Epoch 117, MSE, Train: 216.203 Valid: 226.661
Time elapsed: 0:08:05.308268
Epoch 118, MSE, Train: 216.227 Valid: 224.378
Time elapsed: 0:08:09.406190
Epoch 119, MSE, Train: 216.007 Valid: 219.573
Time elapsed: 0:08:13.494126
Epoch 120, MSE, Train: 216.329 Valid: 220.175
Time elapsed: 0:08:17.577052
Epoch 121, MSE, Train: 216.196 Valid: 222.368
Time elapsed: 0:08:21.663971
Epoch 122, MSE, Train: 215.613 Valid: 220.334
Time elapsed: 0:08:25.732903
Epoch 123, MSE, Train: 215.972 Valid: 219.348
Time elapsed: 0:08:29.828

Time elapsed: 0:03:38.003595
Epoch 52, SMSE, Train: 18.706 Valid: 19.204
Time elapsed: 0:03:42.086521
Epoch 53, SMSE, Train: 18.796 Valid: 19.064
Time elapsed: 0:03:46.349489
Epoch 54, SMSE, Train: 18.620 Valid: 19.435
Time elapsed: 0:03:50.496420
Epoch 55, SMSE, Train: 18.536 Valid: 19.534
Time elapsed: 0:03:54.600361
Epoch 56, SMSE, Train: 18.702 Valid: 18.890
Time elapsed: 0:03:58.741300
Epoch 57, SMSE, Train: 18.677 Valid: 19.115
Time elapsed: 0:04:02.988327
Epoch 58, SMSE, Train: 18.636 Valid: 18.796
Time elapsed: 0:04:07.247293
Epoch 59, SMSE, Train: 18.657 Valid: 18.762
Time elapsed: 0:04:11.355225
Epoch 60, SMSE, Train: 18.755 Valid: 18.987
Time elapsed: 0:04:15.457156
Epoch 61, SMSE, Train: 18.581 Valid: 19.148
Time elapsed: 0:04:19.558086
Epoch 62, SMSE, Train: 18.560 Valid: 19.103
Time elapsed: 0:04:23.810050
Epoch 63, SMSE, Train: 18.673 Valid: 18.701
Time elapsed: 0:04:27.902979
Epoch 64, SMSE, Train: 18.466 Valid: 19.133
Time elapsed: 0:04:32.009911
Epoch 65, SMSE, Train:

Model was trained with the loss function: SMSE
Initial investment 17.68894582328168
Mean Hedging error: -29.255898467657502
CVaR_95: 16.1038, CVaR_99: 31.1355
VaR_95: 7.1293, VaR_99: 21.0139
MSE: 1586.7958, RMSE: 39.8346
Semi-MSE: 19.2353, Semi-RMSE: 4.3858
Skew: -0.8432
 ----------------------- 
Average exposure QDH: -0.0943
 ----------------------- 
Average exposure SQDH: 0.1620


### 8) Training of LSTM - ATM call and puts

In [10]:
nbs_assets          = 2
hedging_instruments = "ATM call and put" 
freq_obs            = "Yearly"

# A) Part 1: MSE
loss_type  = 'MSE'
model_name = 'Two_opts_MSE'
lr         = 0.01   # learning rate of the Adam optimizer
print("Hyperparameters: learning rate=%.4f, LSTM cells=%d, neurons=%d" %(lr, hidden_layers, nbs_units))
LSTM_underlying = DeepAgent(train_two_opts_input.shape[0], batch_size, train_two_opts_input.shape[2], 
        loss_type, nbs_assets, hidden_layers, nbs_units, lr, prepro_risky_assets, hedging_instruments, 
        freq_obs, name = model_name)
    
# Start training 
start = dt.datetime.now()
print('---Training for quadratic deep hedging start---')
with tf.Session() as sess:           
    loss_epoch = LSTM_underlying.training(train_two_opts_input, V_0_batch, disc_batch_year, valid_two_opts_input, 
                                          sess, epochs)
    print('---Training for quadratic deep hedging end---')
    
# Compute measured risk exposure on train, valid and test set with the optimized LSTM
model_predict = DeepAgent(test_two_opts_input.shape[0], batch_size, test_two_opts_input.shape[2], loss_type, 
        nbs_assets, hidden_layers, nbs_units, lr, prepro_risky_assets, hedging_instruments, 
        freq_obs, name = model_name)

with tf.Session() as sess:
    
    # Load the saved model
    model_predict.restore(sess, r"/Users/alexa/Github files/Long-term hedging paper example/Models/%s/models.ckpt" % model_name)

    print("-------------------------------")
    print("Results on the TRAIN SET:")
    # Compute hedging strategies on the training set
    deltas = model_predict.predict(train_two_opts_input, V_0_batch, disc_batch_year, sess, loss_type)    
    hedging_stats(deltas, train_two_opts_input, disc_train_year, V_0_train, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
    print("-------------------------------")
    print("Results on the VALID SET:")
    # Compute hedging strategies on the valid set
    deltas = model_predict.predict(valid_two_opts_input, V_0_batch, disc_batch_year, sess, loss_type)
    hedging_stats(deltas, valid_two_opts_input, disc_valid_year, V_0_valid, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
    print("-------------------------------")
    print("Results on the TEST SET:")
    # Compute hedging strategies on the test set
    deltas_MSE = model_predict.predict(test_two_opts_input, V_0_batch, disc_batch_year, sess, loss_type)
    hedging_stats(deltas_MSE, test_two_opts_input, disc_test_year, V_0_test, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)

print("-------------------------------")
print("-------------------------------")
print("END OF QUADRATIC DEEP HEDGING")
print("-------------------------------")
print("-------------------------------")

# --------------------------------------------------- #
# B) Part 2: SMSE
loss_type  = 'SMSE'
model_name = 'Two_opts_SMSE'
lr         = 0.01/6   # learning rate of the Adam optimizer
print("Hyperparameters: learning rate=%.4f, LSTM cells=%d, neurons=%d" %(lr, hidden_layers, nbs_units))
LSTM_underlying = DeepAgent(train_two_opts_input.shape[0], batch_size, train_two_opts_input.shape[2], 
        loss_type, nbs_assets, hidden_layers, nbs_units, lr, prepro_risky_assets, hedging_instruments, 
        freq_obs, name = model_name)
    
# Start training 
start = dt.datetime.now()
print('---Training for semi-quadratic deep hedging start---')
with tf.Session() as sess:           
    loss_epoch = LSTM_underlying.training(train_two_opts_input, V_0_batch, disc_batch_year, valid_two_opts_input, 
                                          sess, epochs)
    print('---Training for semi-quadratic deep hedging end---')
    
# Compute measured risk exposure on train, valid and test set with the optimized LSTM
model_predict = DeepAgent(test_two_opts_input.shape[0], batch_size, test_two_opts_input.shape[2], loss_type, 
        nbs_assets, hidden_layers, nbs_units, lr, prepro_risky_assets, hedging_instruments, 
        freq_obs, name = model_name)

with tf.Session() as sess:
    
    # Load the saved model
    model_predict.restore(sess, r"/Users/alexa/Github files/Long-term hedging paper example/Models/%s/models.ckpt" % model_name)

    print("-------------------------------")
    print("Results on the TRAIN SET:")
    # Compute hedging strategies on the training set
    deltas = model_predict.predict(train_two_opts_input, V_0_batch, disc_batch_year, sess, loss_type)    
    hedging_stats(deltas, train_two_opts_input, disc_train_year, V_0_train, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
    print("-------------------------------")
    print("Results on the VALID SET:")
    # Compute hedging strategies on the valid set
    deltas = model_predict.predict(valid_two_opts_input, V_0_batch, disc_batch_year, sess, loss_type)
    hedging_stats(deltas, valid_two_opts_input, disc_valid_year, V_0_valid, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
    print("-------------------------------")
    print("Results on the TEST SET:")
    # Compute hedging strategies on the test set
    deltas_SMSE = model_predict.predict(test_two_opts_input, V_0_batch, disc_batch_year, sess, loss_type)
    hedging_stats(deltas_SMSE, test_two_opts_input, disc_test_year, V_0_test, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
# Compute average equity risk exposure
stock_price_unorm       = S_0*np.exp(test_two_opts_input[:,:,0])
portfolio_exposure_QDH  = np.zeros((test_two_opts_input.shape[0]-1, test_two_opts_input.shape[1], 2)) # no exposure at close contract
portfolio_exposure_SQDH = np.zeros((test_two_opts_input.shape[0]-1, test_two_opts_input.shape[1], 2)) # no exposure at close contract 
delta_call_put          = np.zeros((stock_price_unorm.shape[0]-1, stock_price_unorm.shape[1],2)) # deltas across all paths for calls and puts
delta_call_put[:,:,0]   = BS_delta(stock_price_unorm[:-1,:], 1, r, 0.15, stock_price_unorm[:-1,:], 1)  # call deltas 
delta_call_put[:,:,1]   = BS_delta(stock_price_unorm[:-1,:], 1, r, 0.15, stock_price_unorm[:-1,:], -1) # put deltas

portfolio_exposure_QDH[:,:,0]  = deltas_MSE[:,:,0]*delta_call_put[:,:,0]  # call option
portfolio_exposure_QDH[:,:,1]  = deltas_MSE[:,:,1]*delta_call_put[:,:,1]  # put option
    
portfolio_exposure_SQDH[:,:,0] = deltas_SMSE[:,:,0]*delta_call_put[:,:,0]  # call option
portfolio_exposure_SQDH[:,:,1] = deltas_SMSE[:,:,1]*delta_call_put[:,:,1]  # put option

print(" ----------------------- ")
print("Portfolio total exposure - QDH")
print(np.sum(portfolio_exposure_QDH)/(portfolio_exposure_QDH.shape[0]*portfolio_exposure_QDH.shape[1]))   
    
print(" ----------------------- ")
print("Portfolio total exposure - SQDH")
print(np.sum(portfolio_exposure_SQDH)/(portfolio_exposure_SQDH.shape[0]*portfolio_exposure_SQDH.shape[1]))  

Hyperparameters: learning rate=0.0100, LSTM cells=2, neurons=24
---Training for quadratic deep hedging start---
Time elapsed: 0:00:05.345213
Epoch 1, MSE, Train: 112.306 Valid: 27.502
Time elapsed: 0:00:09.829221
Epoch 2, MSE, Train: 25.157 Valid: 23.589
Time elapsed: 0:00:14.312247
Epoch 3, MSE, Train: 21.173 Valid: 19.185
Time elapsed: 0:00:18.787262
Epoch 4, MSE, Train: 20.116 Valid: 18.804
Time elapsed: 0:00:23.272271
Epoch 5, MSE, Train: 19.333 Valid: 17.930
Time elapsed: 0:00:27.790305
Epoch 6, MSE, Train: 19.194 Valid: 17.901
Time elapsed: 0:00:32.129289
Epoch 7, MSE, Train: 18.792 Valid: 21.181
Time elapsed: 0:00:36.463273
Epoch 8, MSE, Train: 18.950 Valid: 18.640
Time elapsed: 0:00:40.801248
Epoch 9, MSE, Train: 18.843 Valid: 18.206
Time elapsed: 0:00:45.135230
Epoch 10, MSE, Train: 19.064 Valid: 19.694
Time elapsed: 0:00:49.481226
Epoch 11, MSE, Train: 18.777 Valid: 21.135
Time elapsed: 0:00:53.827202
Epoch 12, MSE, Train: 18.874 Valid: 18.259
Time elapsed: 0:00:58.159195
Epo

Time elapsed: 0:08:19.343355
Epoch 114, MSE, Train: 16.817 Valid: 16.782
Time elapsed: 0:08:23.709355
Epoch 115, MSE, Train: 17.428 Valid: 17.194
Time elapsed: 0:08:28.069344
Epoch 116, MSE, Train: 16.644 Valid: 17.266
Time elapsed: 0:08:32.391325
Epoch 117, MSE, Train: 16.735 Valid: 17.589
Time elapsed: 0:08:36.739302
Epoch 118, MSE, Train: 16.608 Valid: 16.693
Time elapsed: 0:08:41.112303
Epoch 119, MSE, Train: 16.630 Valid: 16.592
Time elapsed: 0:08:45.466291
Epoch 120, MSE, Train: 16.654 Valid: 17.205
Time elapsed: 0:08:49.809276
Epoch 121, MSE, Train: 17.042 Valid: 16.812
Time elapsed: 0:08:54.148261
Epoch 122, MSE, Train: 16.786 Valid: 16.889
Time elapsed: 0:08:58.470232
Epoch 123, MSE, Train: 16.667 Valid: 17.514
Time elapsed: 0:09:02.799223
Epoch 124, MSE, Train: 16.788 Valid: 17.940
Time elapsed: 0:09:07.136207
Epoch 125, MSE, Train: 16.677 Valid: 16.905
Time elapsed: 0:09:11.468190
Epoch 126, MSE, Train: 16.730 Valid: 16.606
Time elapsed: 0:09:15.799173
Epoch 127, MSE, Train:

Time elapsed: 0:04:18.239719
Epoch 58, SMSE, Train: 1.738 Valid: 1.831
Time elapsed: 0:04:22.616692
Epoch 59, SMSE, Train: 1.749 Valid: 1.907
Time elapsed: 0:04:27.004686
Epoch 60, SMSE, Train: 1.772 Valid: 1.823
Time elapsed: 0:04:31.402684
Epoch 61, SMSE, Train: 1.715 Valid: 1.921
Time elapsed: 0:04:35.765674
Epoch 62, SMSE, Train: 1.724 Valid: 1.810
Time elapsed: 0:04:40.145667
Epoch 63, SMSE, Train: 1.734 Valid: 1.963
Time elapsed: 0:04:44.531662
Epoch 64, SMSE, Train: 1.739 Valid: 1.878
Time elapsed: 0:04:48.931661
Epoch 65, SMSE, Train: 1.713 Valid: 1.825
Time elapsed: 0:04:53.322657
Epoch 66, SMSE, Train: 1.712 Valid: 1.871
Time elapsed: 0:04:57.877764
Epoch 67, SMSE, Train: 1.709 Valid: 1.788
Time elapsed: 0:05:02.233753
Epoch 68, SMSE, Train: 1.697 Valid: 1.858
Time elapsed: 0:05:06.616739
Epoch 69, SMSE, Train: 1.734 Valid: 1.903
Time elapsed: 0:05:11.016745
Epoch 70, SMSE, Train: 1.679 Valid: 2.003
Time elapsed: 0:05:15.581773
Epoch 71, SMSE, Train: 1.696 Valid: 1.748
Time e

### 9) Training of LSTM - Six options

In [11]:
nbs_assets          = 6
hedging_instruments = "Six options"
freq_obs            = "Yearly"

# A) Part 1: MSE
loss_type  = 'MSE'
model_name = 'Six_opts_MSE'
lr         = 0.01   # learning rate of the Adam optimizer
print("Hyperparameters: learning rate=%.4f, LSTM cells=%d, neurons=%d" %(lr, hidden_layers, nbs_units))
LSTM_underlying = DeepAgent(train_six_opts_input.shape[0], batch_size, train_six_opts_input.shape[2], 
        loss_type, nbs_assets, hidden_layers, nbs_units, lr, prepro_risky_assets, hedging_instruments, 
        freq_obs, name = model_name)
    
# Start training 
start = dt.datetime.now()
print('---Training for quadratic deep hedging start---')
with tf.Session() as sess:           
    loss_epoch = LSTM_underlying.training(train_six_opts_input, V_0_batch, disc_batch_year, valid_six_opts_input, 
                                          sess, epochs)
    print('---Training for quadratic deep hedging end---')
    
# Compute measured risk exposure on train, valid and test set with the optimized LSTM
model_predict = DeepAgent(test_six_opts_input.shape[0], batch_size, test_six_opts_input.shape[2], loss_type, 
        nbs_assets, hidden_layers, nbs_units, lr, prepro_risky_assets, hedging_instruments, 
        freq_obs, name = model_name)

with tf.Session() as sess:
    
    # Load the saved model
    model_predict.restore(sess, r"/Users/alexa/Github files/Long-term hedging paper example/Models/%s/models.ckpt" % model_name)

    print("-------------------------------")
    print("Results on the TRAIN SET:")
    # Compute hedging strategies on the training set
    deltas = model_predict.predict(train_six_opts_input, V_0_batch, disc_batch_year, sess, loss_type)    
    hedging_stats(deltas, train_six_opts_input, disc_train_year, V_0_train, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
    print("-------------------------------")
    print("Results on the VALID SET:")
    # Compute hedging strategies on the valid set
    deltas = model_predict.predict(valid_six_opts_input, V_0_batch, disc_batch_year, sess, loss_type)
    hedging_stats(deltas, valid_six_opts_input, disc_valid_year, V_0_valid, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
    print("-------------------------------")
    print("Results on the TEST SET:")
    # Compute hedging strategies on the test set
    deltas_MSE = model_predict.predict(test_six_opts_input, V_0_batch, disc_batch_year, sess, loss_type)
    hedging_stats(deltas_MSE, test_six_opts_input, disc_test_year, V_0_test, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)

print("-------------------------------")
print("-------------------------------")
print("END OF QUADRATIC DEEP HEDGING")
print("-------------------------------")
print("-------------------------------")

# --------------------------------------------------- #
# B) Part 2: SMSE
loss_type  = 'SMSE'
model_name = 'Six_opts_SMSE'
lr         = 0.01/6   # learning rate of the Adam optimizer
print("Hyperparameters: learning rate=%.4f, LSTM cells=%d, neurons=%d" %(lr, hidden_layers, nbs_units))
LSTM_underlying = DeepAgent(train_six_opts_input.shape[0], batch_size, train_six_opts_input.shape[2], 
        loss_type, nbs_assets, hidden_layers, nbs_units, lr, prepro_risky_assets, hedging_instruments, 
        freq_obs, name = model_name)
    
# Start training 
start = dt.datetime.now()
print('---Training for semi-quadratic deep hedging start---')
with tf.Session() as sess:           
    loss_epoch = LSTM_underlying.training(train_six_opts_input, V_0_batch, disc_batch_year, valid_six_opts_input, 
                                          sess, epochs)
    print('---Training for semi-quadratic deep hedging end---')
    
# Compute measured risk exposure on train, valid and test set with the optimized LSTM
model_predict = DeepAgent(test_six_opts_input.shape[0], batch_size, test_six_opts_input.shape[2], loss_type, 
        nbs_assets, hidden_layers, nbs_units, lr, prepro_risky_assets, hedging_instruments, 
        freq_obs, name = model_name)

with tf.Session() as sess:
    
    # Load the saved model
    model_predict.restore(sess, r"/Users/alexa/Github files/Long-term hedging paper example/Models/%s/models.ckpt" % model_name)

    print("-------------------------------")
    print("Results on the TRAIN SET:")
    # Compute hedging strategies on the training set
    deltas = model_predict.predict(train_six_opts_input, V_0_batch, disc_batch_year, sess, loss_type)    
    hedging_stats(deltas, train_six_opts_input, disc_train_year, V_0_train, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
    print("-------------------------------")
    print("Results on the VALID SET:")
    # Compute hedging strategies on the valid set
    deltas = model_predict.predict(valid_six_opts_input, V_0_batch, disc_batch_year, sess, loss_type)
    hedging_stats(deltas, valid_six_opts_input, disc_valid_year, V_0_valid, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
    print("-------------------------------")
    print("Results on the TEST SET:")
    # Compute hedging strategies on the test set
    deltas_SMSE = model_predict.predict(test_six_opts_input, V_0_batch, disc_batch_year, sess, loss_type)
    hedging_stats(deltas_SMSE, test_six_opts_input, disc_test_year, V_0_test, loss_type, model_name, prepro_risky_assets, 
                  hedging_instruments, freq_obs)
    
# Compute average equity risk exposure
stock_price_unorm       = S_0*np.exp(test_six_opts_input[:,:,0])
portfolio_exposure_QDH  = np.zeros((test_six_opts_input.shape[0]-1, test_six_opts_input.shape[1], 6))      # no exposure at close contract
portfolio_exposure_SQDH = np.zeros((test_six_opts_input.shape[0]-1, test_six_opts_input.shape[1], 6))      # no exposure at close contract 
delta_call_put          = np.zeros((stock_price_unorm.shape[0]-1, stock_price_unorm.shape[1],6))           # deltas across all paths for calls and puts
delta_call_put[:,:,0]   = BS_delta(stock_price_unorm[:-1,:], 1, r, 0.15, stock_price_unorm[:-1,:], 1)      # ATM call deltas 
delta_call_put[:,:,1]   = BS_delta(stock_price_unorm[:-1,:], 1, r, 0.15, 1.1*stock_price_unorm[:-1,:], 1)  # OTM call deltas
delta_call_put[:,:,2]   = BS_delta(stock_price_unorm[:-1,:], 1, r, 0.15, 1.2*stock_price_unorm[:-1,:], 1)  # DOTM call deltas
delta_call_put[:,:,3]   = BS_delta(stock_price_unorm[:-1,:], 1, r, 0.15, stock_price_unorm[:-1,:], -1)     # ATM call deltas 
delta_call_put[:,:,4]   = BS_delta(stock_price_unorm[:-1,:], 1, r, 0.15, 0.9*stock_price_unorm[:-1,:], -1) # OTM call deltas
delta_call_put[:,:,5]   = BS_delta(stock_price_unorm[:-1,:], 1, r, 0.15, 0.8*stock_price_unorm[:-1,:], -1) # DOTM call deltas

portfolio_exposure_QDH[:,:,0]  = deltas_MSE[:,:,0]*delta_call_put[:,:,0]  # ATM call option
portfolio_exposure_QDH[:,:,1]  = deltas_MSE[:,:,1]*delta_call_put[:,:,1]  # OTM call option
portfolio_exposure_QDH[:,:,2]  = deltas_MSE[:,:,2]*delta_call_put[:,:,2]  # DOTM call option
portfolio_exposure_QDH[:,:,3]  = deltas_MSE[:,:,3]*delta_call_put[:,:,3]  # ATM put option
portfolio_exposure_QDH[:,:,4]  = deltas_MSE[:,:,4]*delta_call_put[:,:,4]  # OTM put option
portfolio_exposure_QDH[:,:,5]  = deltas_MSE[:,:,5]*delta_call_put[:,:,5]  # DOTM put option

portfolio_exposure_SQDH[:,:,0]  = deltas_SMSE[:,:,0]*delta_call_put[:,:,0]  # ATM call option
portfolio_exposure_SQDH[:,:,1]  = deltas_SMSE[:,:,1]*delta_call_put[:,:,1]  # OTM call option
portfolio_exposure_SQDH[:,:,2]  = deltas_SMSE[:,:,2]*delta_call_put[:,:,2]  # DOTM call option
portfolio_exposure_SQDH[:,:,3]  = deltas_SMSE[:,:,3]*delta_call_put[:,:,3]  # ATM put option
portfolio_exposure_SQDH[:,:,4]  = deltas_SMSE[:,:,4]*delta_call_put[:,:,4]  # OTM put option
portfolio_exposure_SQDH[:,:,5]  = deltas_SMSE[:,:,5]*delta_call_put[:,:,5]  # DOTM put option

print(" ----------------------- ")
print("Portfolio total exposure - QDH")
print(np.sum(portfolio_exposure_QDH)/(portfolio_exposure_QDH.shape[0]*portfolio_exposure_QDH.shape[1]))   
    
print(" ----------------------- ")
print("Portfolio total exposure - SQDH")
print(np.sum(portfolio_exposure_SQDH)/(portfolio_exposure_SQDH.shape[0]*portfolio_exposure_SQDH.shape[1]))   

Hyperparameters: learning rate=0.0100, LSTM cells=2, neurons=24
---Training for quadratic deep hedging start---
Time elapsed: 0:00:06.015364
Epoch 1, MSE, Train: 114.817 Valid: 15.215
Time elapsed: 0:00:11.174534
Epoch 2, MSE, Train: 9.650 Valid: 7.839
Time elapsed: 0:00:16.346708
Epoch 3, MSE, Train: 7.087 Valid: 6.382
Time elapsed: 0:00:21.518890
Epoch 4, MSE, Train: 6.158 Valid: 5.425
Time elapsed: 0:00:26.674059
Epoch 5, MSE, Train: 5.561 Valid: 3.825
Time elapsed: 0:00:31.836231
Epoch 6, MSE, Train: 4.545 Valid: 3.301
Time elapsed: 0:00:37.015406
Epoch 7, MSE, Train: 4.162 Valid: 3.067
Time elapsed: 0:00:42.042546
Epoch 8, MSE, Train: 3.694 Valid: 6.040
Time elapsed: 0:00:47.053674
Epoch 9, MSE, Train: 3.749 Valid: 3.191
Time elapsed: 0:00:52.171845
Epoch 10, MSE, Train: 3.079 Valid: 2.689
Time elapsed: 0:00:57.172979
Epoch 11, MSE, Train: 3.096 Valid: 3.009
Time elapsed: 0:01:02.211113
Epoch 12, MSE, Train: 2.705 Valid: 3.368
Time elapsed: 0:01:07.389296
Epoch 13, MSE, Train: 2.7

Time elapsed: 0:09:38.355357
Epoch 117, MSE, Train: 1.176 Valid: 1.846
Time elapsed: 0:09:43.309472
Epoch 118, MSE, Train: 1.133 Valid: 1.659
Time elapsed: 0:09:48.209592
Epoch 119, MSE, Train: 1.291 Valid: 1.126
Time elapsed: 0:09:53.091700
Epoch 120, MSE, Train: 1.137 Valid: 1.213
Time elapsed: 0:09:57.983801
Epoch 121, MSE, Train: 1.162 Valid: 1.064
Time elapsed: 0:10:02.870918
Epoch 122, MSE, Train: 1.150 Valid: 1.125
Time elapsed: 0:10:07.770021
Epoch 123, MSE, Train: 1.198 Valid: 1.432
Time elapsed: 0:10:12.659139
Epoch 124, MSE, Train: 9.108 Valid: 4.537
Time elapsed: 0:10:17.550241
Epoch 125, MSE, Train: 3.557 Valid: 2.011
Time elapsed: 0:10:22.425355
Epoch 126, MSE, Train: 1.851 Valid: 1.647
Time elapsed: 0:10:27.307462
Epoch 127, MSE, Train: 1.580 Valid: 1.394
Time elapsed: 0:10:32.187570
Epoch 128, MSE, Train: 1.488 Valid: 1.250
Time elapsed: 0:10:37.112678
Epoch 129, MSE, Train: 1.477 Valid: 1.215
Time elapsed: 0:10:42.014799
Epoch 130, MSE, Train: 1.340 Valid: 1.231
Time e

Time elapsed: 0:05:08.697088
Epoch 62, SMSE, Train: 0.136 Valid: 0.118
Time elapsed: 0:05:13.758236
Epoch 63, SMSE, Train: 0.113 Valid: 0.111
Time elapsed: 0:05:18.670350
Epoch 64, SMSE, Train: 0.134 Valid: 0.140
Time elapsed: 0:05:23.595459
Epoch 65, SMSE, Train: 0.127 Valid: 0.185
Time elapsed: 0:05:28.653615
Epoch 66, SMSE, Train: 0.136 Valid: 0.104
Time elapsed: 0:05:33.561719
Epoch 67, SMSE, Train: 0.155 Valid: 0.151
Time elapsed: 0:05:38.502850
Epoch 68, SMSE, Train: 0.125 Valid: 0.105
Time elapsed: 0:05:43.399961
Epoch 69, SMSE, Train: 0.106 Valid: 0.111
Time elapsed: 0:05:48.295063
Epoch 70, SMSE, Train: 0.124 Valid: 0.165
Time elapsed: 0:05:53.176179
Epoch 71, SMSE, Train: 0.175 Valid: 0.112
Time elapsed: 0:05:58.073290
Epoch 72, SMSE, Train: 0.099 Valid: 0.109
Time elapsed: 0:06:02.986404
Epoch 73, SMSE, Train: 0.114 Valid: 0.106
Time elapsed: 0:06:07.890517
Epoch 74, SMSE, Train: 0.113 Valid: 0.154
Time elapsed: 0:06:12.948697
Epoch 75, SMSE, Train: 0.125 Valid: 0.098
Time e